In [113]:
#!/usr/bin/env python3
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style

# seria interessante ter uma selecao das acoes pra escolher, com bastante variedade
# e arrumar algum jeito de fazer as regras de quantias máximas ou mínimas para os tipos
# diferentes das acoes, por exemplo, 30% de energia, 30% de bancos e 40% tecnologias...

testCSV = './test_market_returns_brl.csv'
stockRows = 1243
stockCols = 63

dfStock = pd.read_csv(testCSV, nrows=stockRows)
dfStock.head()

,Date,^BVSP,ABEV3.SA,ASAI3.SA,AZUL4.SA,B3SA3.SA,BBSE3.SA,BBDC4.SA,BBDC3.SA,BRAP4.SA,...,QUAL3.SA,RADL3.SA,RAIL3.SA,SBSP3.SA,TOTS3.SA,UGPA3.SA,USIM5.SA,VALE3.SA,WEGE3.SA,YDUQ3.SA
0,2017-01-02,59589.0,13.968292,NaN,NaN,4.515615,18.234648,11.856694,12.102320,7.427600,...,14.848257,11.654523,6.11,24.308306,7.483278,29.983891,3.654238,18.692446,5.076362,13.344676
1,2017-01-03,61814.0,14.250913,NaN,NaN,4.701455,18.883545,12.358868,12.592777,7.740072,...,15.495207,12.100480,6.18,24.591063,7.867117,30.469639,3.842324,19.520405,5.331861,14.087488
2,2017-01-04,61589.0,14.139578,NaN,NaN,4.790214,19.480005,12.280593,12.588573,7.571030,...,15.629331,11.929400,6.18,25.302233,7.819534,29.983891,4.075193,19.169830,5.204111,14.018389
3,2017-01-05,62071.0,14.216655,NaN,NaN,4.731965,19.086735,12.416537,12.660000,7.950093,...,15.534655,11.750633,6.19,25.370777,7.860775,30.160522,4.119975,19.900818,5.190664,13.793818
4,2017-01-06,61665.0,14.096756,NaN,NaN,4.729191,18.791780,12.494811,12.756644,7.811786,...,15.376862,11.775620,6.35,25.002340,7.616511,30.028051,4.003540,19.371222,5.234367,13.690170


In [114]:
# cleaning the data

dfStock = dfStock.dropna(axis=0)
stockData = dfStock.iloc[0:, 20:32]
marketData = dfStock.iloc[0:, [1]]
# stockData = stockData.dropna(axis=0)
# stockData.head()

# update das cols e rows depois da limpeza
[stockRows, stockCols] = stockData.shape
# print(stockRows, stockCols)
# [marketRows, marketCols]=marketData.shape
# print(marketRows, marketCols)

In [115]:
# function computes asset returns 
def StockReturnsComputing(StockPrice, Rows, Columns):
    
    import numpy as np
    
    StockReturn = np.zeros([Rows-1, Columns])
    for j in range(Columns):        # j: Assets
        for i in range(Rows-1):     # i: Daily Prices
            StockReturn[i,j]=((StockPrice[i+1, j]-StockPrice[i,j])/StockPrice[i,j])*100

    return StockReturn


In [116]:
# pegando as labels das stocks
stockLabels = dfStock.columns[2:stockCols + 1].tolist()

stockPricesArr = np.asarray(stockData)
[rows, cols] = stockPricesArr.shape

# compute stock returns
stockReturnsArr = StockReturnsComputing(stockPricesArr, rows, cols)

# compute market returns
marketPricesArr = np.asarray(marketData)
[mRows, mCols] = marketPricesArr.shape
marketReturnsArr = StockReturnsComputing(marketPricesArr, mRows, mCols)
marketReturnsArr



array([[ 1.09212852],
       [-0.31916801],
       [ 1.35451144],
       [ 2.22912415],
       [-3.98430583],
       [ 0.65001989],
       [ 1.29793139],
       [ 1.95786338],
       [-0.71662144],
       [ 0.60529082],
       [-0.72441685],
       [ 2.21892842],
       [-1.47062609],
       [ 1.20781992],
       [-1.06950491],
       [-1.49331617],
       [-1.05772457],
       [ 1.50449743],
       [ 0.90637363],
       [ 0.55584112],
       [ 1.23983053],
       [-0.18485237],
       [-1.18404582],
       [ 1.96524169],
       [-0.01616774],
       [ 0.10638388],
       [ 0.58576481],
       [-0.54347367],
       [ 0.97051075],
       [ 0.40820793],
       [ 0.83656756],
       [ 0.33750364],
       [ 0.34216784],
       [-0.14862031],
       [-0.72105446],
       [-0.57553597],
       [ 0.97092259],
       [ 0.05392848],
       [-1.00087068],
       [ 1.39461252],
       [-0.81534534],
       [-0.9761298 ],
       [ 0.26494188],
       [-1.25577767],
       [ 1.57333152],
       [ 0

In [117]:
# calculo do beta pras stocks
beta = []
marketVar = np.var(marketReturnsArr, ddof=1)

for i in range(12):
    CovarMat = np.cov(marketReturnsArr[:,0], stockReturnsArr[:, i ])
    Covar  = CovarMat[1,0]
    beta.append(Covar/marketVar)
    
#display results
print('Asset Betas:\n')
for data in beta:
    print('{:9.3f}'.format(data))
    
print(len(beta))


Asset Betas:

    1.649
    1.280
    1.177
    1.109
    1.496
    0.547
    0.822
    0.958
    0.764
    1.646
    0.747
    0.971
12


In [118]:
# consideramos que as stocks com Beta > 1 são as mais voláteis, temos a seguinte separação entre elas
beta_12_stocks = [0.779, 0.703, 1.743, 1.250, 0.734, 1.168, 1.173, 0.639, 1.016, 1.157, 0.730, 0.784]
beta_20_32_stocks = [1.649, 1.280, 1.177, 1.109, 1.496, 0.547, 0.822, 0.958, 0.764, 1.646, 0.747, 0.971]